In [ ]:
### Import of modules

import pandas as pd
import openpyxl
import time
import datetime
import os
from pandas import Timestamp
from openpyxl.utils.cell import get_column_letter

full_running_time = time.perf_counter()

In [ ]:
##### Configuration and initial definitions #####

### Configuration: How many users/rows do we have in the seed list?
### 438

file_range = 438


### Where the files are stored (All + specifically the source data)
dataSource = r'C:\\Users\\Luis\\Desktop\\tweeting_politics\\'
dataSourceFiles = dataSource + r'source files'

### Definition of excel file and locations

finalfile = openpyxl.load_workbook(r'C:\\Users\\Luis\\Desktop\\tweeting_politics\\' + 'Seedlist.xlsx')

sheet = finalfile['General']
timelines = finalfile['Month']
sourcestab = finalfile['Sources']
timing = finalfile['Hour']
weekDay = finalfile['Week']
replies = finalfile['Replies']
retweets = finalfile['Retweets']
hashtags = finalfile['Hashtags']
quotes = finalfile['Quotes']

### Columns for each data field in final excel

sheet_cell_locs = {
                    "partycell": "D",
                    "namecell": "J",
                    "seating": "F",
                    "leaving": "G",
                    "followerscell": "O",
                    "tweetcell": "R",
                    "retweetcell": "S",
                    "replycell": "T",
                    "quotecell": "U",
                    "mediacell": "V",      
                    "replyratecell": "AA",  
                    "nlcell": "W",
                    "frcell": "X",
                    "encell": "Y",
                    "othercell": "Z",
                    "usercreationcell": "M",
                    "avretweetcell": "P",    
                    "oldtweets": "AF",
                    "newtweets": "AG",
                    "partytweets": "AB",     
                    "avfavcell": "Q",   
                    "friendscell": "AC",
                    "verified": "N",
                    "favouritescell": "AD",  
                    "textlenghtcell": "AE"           
}

### Twitter accounts of political parties

partytwitters = [
    'agora_brussels',
    'cdenv', 'CDenV_VlaParl', 'cdenv_Kamer', 'CDenV_Brussel',
    'defi_eu', 'DefiParlBxl', 
    'Ecolo', 'EcoloBruxelles', 
    'groen', 'groenbrussel', 
    'LesEngages_be', 'LesEngages_BXL', 'LesEngages_fed', 'LesEngages_Wal',
    'mr_officiel', 'mr_pfwb', 'MRWallon', 'MRBruxelles', 
    'de_NVA', 'NVA_VlaParl', 'NVA_Brussel', 
    'openvld', 'OpenVldVlaParl', 'OpenVldBrussel', 
    'psofficiel', 'pslachambre', 'Socialistes_PW', 'roupepsprb', 'psbruxellois', 
    'ptbbelgique', 'PTBlachambre', 'PTBparlbru', 
    'PvdA', 'PVDAinVlaParl', 'pvdabelgie', 'PVDA_BruParl', 'PvdABrussel', 
    'vlbelang', 'VB_Kamer', 'VB_VlaParl', 'VlaamsBelangBru', 
    'vooruit_nu'  
]

In [ ]:
##### Import and initial processing of files #####


### Import of source files and consolidation in a single dataframe
sourceFiles = []
for root, dirs, file in os.walk(dataSourceFiles):
    for source in file:
        sourceFiles.append(root + r'\\' + source)
for i in range(len(sourceFiles)):
    sourceFiles[i] = pd.read_csv(sourceFiles[i])
data = pd.concat(sourceFiles)
print(data.shape)

### user screen names normalised in lower caps for later
data["user_screen_name"] = data["user_screen_name"].str.lower()

### Split of data per period
olddata = data[((data['parsed_created_at'] <= '2019-06'))]
newdata = data[((data['parsed_created_at'] >= '2022-05'))]
data = data[((data['parsed_created_at'] >= '2019-06') & (data['parsed_created_at'] <= '2022-05'))]

### Adaptation: New column with hour of publication and day of week

data = data.astype({'parsed_created_at': 'datetime64'})
data['tweetHour'] = data.parsed_created_at.apply(lambda x : x.hour)

# Monday = 0, Sunday = 6
data['DayOfWeek'] = data.parsed_created_at.apply(lambda x : x.weekday())

### Normalisation of dates

data['user_created_at'] = pd.to_datetime(data['user_created_at'], format='%a %b %d %X %z %Y')

In [ ]:
##### Recallable functions #####


### Finding the right cell for each data point
def sheet_cell(column, row):
    return sheet_cell_locs[column] + str(row + 2)

In [ ]:
##### Further split of data to account for representatives entering or leaving the chambers at a different point
print(data.shape)
testdata = data
print(testdata.shape)

for i in range(file_range):
    if sheet[sheet_cell("namecell",i)].value != 'Null' and sheet[sheet_cell("namecell",i)].value:
        if type(sheet[sheet_cell("seating",i)].value) == datetime.datetime:
            print(sheet[sheet_cell("namecell",i)].value)
            print(len(testdata[((testdata['user_screen_name'] == sheet[sheet_cell("namecell",i)].value.lower()))]))
            startdate = sheet[sheet_cell("seating",i)].value
            testdata = testdata[-((testdata['user_screen_name'] == sheet[sheet_cell("namecell",i)].value.lower()) & (testdata['parsed_created_at'] <= startdate))]
            print(len(testdata[((testdata['user_screen_name'] == sheet[sheet_cell("namecell",i)].value.lower()))]))

print(testdata.shape)

for i in range(file_range):
    if sheet[sheet_cell("namecell",i)].value != 'Null' and sheet[sheet_cell("namecell",i)].value:
        if type(sheet[sheet_cell("leaving",i)].value) == datetime.datetime:
            print(sheet[sheet_cell("namecell",i)].value)
            print(len(testdata[((testdata['user_screen_name'] == sheet[sheet_cell("namecell",i)].value.lower()))]))
            startdate = sheet[sheet_cell("leaving",i)].value
            testdata = testdata[-((testdata['user_screen_name'] == sheet[sheet_cell("namecell",i)].value.lower()) & (testdata['parsed_created_at'] >= startdate))]
            print(len(testdata[((testdata['user_screen_name'] == sheet[sheet_cell("namecell",i)].value.lower()))]))


print(testdata.shape)



data = testdata


In [ ]:
##### Creation of lists and dictionaries to be used later #####

### Data source - Platform or device from which the user posted

# list of all representatives present in dataset
allparliaments = data.user_screen_name.unique()

# list of all data sources for the tweets present in the data, and another one for the lesser ones
sources = data.source.unique().tolist()
altsources = []

# removing data sources with less than 500 uses from list a to list b
for i in range(len(sources)):
    if len(data[(data['source'] == sources[i])]) < 500:
        if sources[i] not in altsources:
            altsources.append(sources[i])
        sources[i] = 'null'
while 'null' in sources:
    sources.remove('null')   

    
### Definition: Dictionary with all members and their political parties
### Definition: List and dictionary with the names of the political parties

userparties = {}
for i in range(file_range):
    if sheet[sheet_cell("namecell",i)].value != 'Null' and sheet[sheet_cell("namecell",i)].value:
        userparties[sheet[sheet_cell("namecell",i)].value] = sheet[sheet_cell("partycell",i)].value     
        
allPartiesDict = {}

for i in userparties:
    if userparties[i] not in allPartiesDict:
        allPartiesDict[userparties[i]] = 0
    
    
### Definition: List of months covered by analysis for segmentation of periods
    
coveredMonths = []
month, year = 0, 0

for i in range(36):
    if month == 0 and year == 0:
        month = 6
        realYear = 2019
        realMonth = 6
    else:
        if month >= 12:
            year += 1
            month = 1
        else:
            month += 1
        realYear = 2019 + year
        realMonth = 0 +  month
    if realMonth < 10:
        date = str(realYear) + '-' + '0' + str(realMonth)
    else:
        date = str(realYear) + '-' + str(realMonth)
    coveredMonths.append(date)

In [ ]:
### Execution and gathering of data for the main dashboard sheet

full_running_time = time.perf_counter()

for user in range(file_range):
    full_running_time = time.perf_counter()
    thisUser = sheet[sheet_cell("namecell",user)].value.lower()
    if sheet[sheet_cell("namecell",user)].value != 'Null' and sheet[sheet_cell("namecell",user)].value != 'Screen name' and sheet[sheet_cell("namecell",user)].value:
        print(str(user + 1) + ' - ' + thisUser + ': ' + str(len(data[(data['user_screen_name'] == thisUser)])))
        sheet[sheet_cell("followerscell",user)] = data.loc[data['user_screen_name'] == thisUser]['user_followers_count'].min()
        if len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower())]) >= 1:
            sheet[sheet_cell("tweetcell",user)] = len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['tweet_type'] == 'original')])
            sheet[sheet_cell("retweetcell",user)] = len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['tweet_type'] == 'retweet')])
            sheet[sheet_cell("replycell",user)] = len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['tweet_type'] == 'reply')])
            sheet[sheet_cell("quotecell",user)] = len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['tweet_type'] == 'quote')])                        
            sheet[sheet_cell("usercreationcell",user)] = Timestamp.date(data.loc[data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()]['user_created_at'].min())
            sheet[sheet_cell("oldtweets",user)] = len(olddata[(olddata['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower())])
            sheet[sheet_cell("newtweets",user)] = len(newdata[(newdata['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower())])   
            sheet[sheet_cell("verified",user)] = data.loc[data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()]['user_verified'].min()         
            sheet[sheet_cell("nlcell",user)] = len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['lang'] == 'nl')])
            sheet[sheet_cell("frcell",user)] = len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['lang'] == 'fr')])
            sheet[sheet_cell("encell",user)] = len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['lang'] == 'en')])
            sheet[sheet_cell("othercell",user)] = len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['lang'] != 'nl') & (data['lang'] != 'fr') & (data['lang'] != 'en')])      
            sheet[sheet_cell("avretweetcell",user)] = data.loc[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['tweet_type'] == 'original')]['retweet_count'].mean()
            sheet[sheet_cell("mediacell",user)] = len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['media']) & (data['tweet_type'] == 'original')])
            sheet[sheet_cell("replyratecell",user)] = len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['in_reply_to_screen_name'].isin(allparliaments))])
            sheet[sheet_cell("partytweets",user)] = len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['retweet_or_quote_screen_name'].isin(partytwitters))])           
            sheet[sheet_cell("textlenghtcell",user)] = data.loc[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['tweet_type'] == 'original')]['text'].apply(len).mean()
            sheet[sheet_cell("avfavcell",user)] = data.loc[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['tweet_type'] == 'original')]['favorite_count'].mean()
        sheet[sheet_cell("friendscell",user)] = data.loc[data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()]['user_friends_count'].max()
        sheet[sheet_cell("favouritescell",user)] = data.loc[data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()]['user_favourites_count'].max()
    else:
        print("Missing user: " + str(user + 1) + ' - ' + thisUser)
    print('Full execution time: ' + str(time.perf_counter() - full_running_time))
finalfile.save(dataSource + 'Seedlist.xlsx')



In [ ]:
### Collection: No of tweets month by month - Timelines

# Note: Seems to have some mistake. The data is collected one row above the right one, and so the last user has no data.

for user in range(file_range):
    if sheet[sheet_cell("namecell",user)].value != 'Null' and sheet[sheet_cell("namecell",user)].value != 'Screen name' and sheet[sheet_cell("namecell",user)].value:
        print(str(user) + ' - ' + sheet[sheet_cell("namecell",user)].value + ': ' + str(len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower())])))
        for i in range(len(coveredMonths)-1):
            monthtomonth = get_column_letter(i+8) + str(user+2)
            timelines[monthtomonth] = len(data[((data['parsed_created_at'] >= coveredMonths[i]) & (data['parsed_created_at'] <= coveredMonths[i+1]) & (data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()))])
 
        
finalfile.save(dataSource + 'Seedlist.xlsx')

In [ ]:
### Collection: Tweet source - Sources
### Note: Long running time

for user in range(file_range):
    if sheet[sheet_cell("namecell",user)].value != 'Null' and sheet[sheet_cell("namecell",user)].value != 'Screen name' and sheet[sheet_cell("namecell",user)].value:
        print(str(user) + ' - ' + str(sheet[sheet_cell("namecell",user)].value) + ': ' + str(len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower())])))
        if len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower())]) >= 1:
            for i in range(len(sources)):
                if len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['source'] == sources[i])]) > 0:
                    sourcecell = get_column_letter(i+8) + str(user+2)
                    sourcestab[sourcecell] = len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['source'] == sources[i])])
                    print(sourcestab[sourcecell])
                    print(sourcestab[sourcecell].value)
            other = 0
            for i in range(len(altsources)):
                if len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['source'] == altsources[i])]) > 0:
                    sourcecell = 'Z' + str(user+2)
                    other += len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['source'] == altsources[i])])
                    sourcestab[sourcecell] = other
                    print(sourcestab[sourcecell])
                    print(sourcestab[sourcecell].value)
                    
                    
finalfile.save(dataSource + 'Seedlist.xlsx')

In [ ]:
### retweet_or_quote_screen_name - retweet of same vs other parties - Retweets
### Note: This code has to go through many loops so the assignation of 'members[member]' can take up to one minute per user (438) in total
full_running_time = time.perf_counter()

x = 0
for i in allPartiesDict:
    retweets[str(get_column_letter(x+8) + '1')] = i
    x += 1
    
finalfile.save(dataSource + 'Seedlist.xlsx')

for user in range(file_range):
    print('A: ' + str(time.perf_counter() - full_running_time))
    full_running_time = time.perf_counter()
    if sheet[sheet_cell("namecell",user)].value != 'Null' and sheet[sheet_cell("namecell",user)].value != 'Screen name' and sheet[sheet_cell("namecell",user)].value:
        retweetsPerParty = allPartiesDict.copy()
        print(str(user) + ' - ' + sheet[sheet_cell("namecell",user)].value + ': ' + str(len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['tweet_type'] == 'retweet')])))
        if len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['tweet_type'] == 'retweet')]) > 0:
            members = {}
            for member in userparties:
                if member.lower() != sheet[sheet_cell("namecell",user)].value.lower():
                    members[member] = len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['retweet_or_quote_screen_name'] == member) & (data['tweet_type'] == 'retweet')])
            for member in members:
                if members[member] > 0:
                    print(member + ' - ' + userparties[member] + ' - ' + str(members[member]))
                    retweetsPerParty[userparties[member]] += members[member]
        x = 7
        for i in retweetsPerParty:
            x += 1
            partyCell = get_column_letter(x) + str(user+2)
            retweets[partyCell] = retweetsPerParty[i]
            print(partyCell + ' - ' + str(retweetsPerParty[i]))
                        
finalfile.save(dataSource + 'Seedlist.xlsx')



In [ ]:
### retweet_or_quote_screen_name - quotes of same vs other parties - Quotes
### Note: This code has to go through many loops so the assignation of 'members[member]' can take up to one minute per user (438) in total

x = 0
for i in allPartiesDict:
    quotes[str(get_column_letter(x+8) + '1')] = i
    x += 1
    
finalfile.save(dataSource + 'Seedlist.xlsx')

for user in range(file_range):
    print('A: ' + str(time.perf_counter() - full_running_time))
    full_running_time = time.perf_counter()
    if sheet[sheet_cell("namecell",user)].value != 'Null' and sheet[sheet_cell("namecell",user)].value != 'Screen name' and sheet[sheet_cell("namecell",user)].value:
        quotesPerParty = allPartiesDict.copy()
        print(str(user) + ' - ' + sheet[sheet_cell("namecell",user)].value + ': ' + str(len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['tweet_type'] == 'quote')])))
        if len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['tweet_type'] == 'quote')]) > 0:
            members = {}
            for member in userparties:
                if member.lower() != sheet[sheet_cell("namecell",user)].value.lower():
                    members[member] = len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['retweet_or_quote_screen_name'] == member) & (data['tweet_type'] == 'quote')])
            for member in members:
                if members[member] > 0:
                    print(member + ' - ' + userparties[member] + ' - ' + str(members[member]))
                    quotesPerParty[userparties[member]] += members[member]
        x = 7
        for i in quotesPerParty:
            x += 1
            partyCell = get_column_letter(x) + str(user+2)
            quotes[partyCell] = quotesPerParty[i]
            print(partyCell + ' - ' + str(quotesPerParty[i]))
                        
finalfile.save(dataSource + 'Seedlist.xlsx')


In [ ]:
### in_reply_to_screen_name - Reply to same vs other parties - Replies
### Note: long running time

full_running_time = time.perf_counter()

# Creates top row with names of parties to be filled in
x = 0
for i in allPartiesDict:
    replies[str(get_column_letter(x+8) + '1')] = i
    x += 1
finalfile.save(dataSource + 'Seedlist.xlsx')

print('A: ' + str(time.perf_counter() - full_running_time))


# Fills in data with each individual user
for user in range(file_range):
    full_running_time = time.perf_counter()
    if sheet[sheet_cell("namecell",user)].value != 'Null' and sheet[sheet_cell("namecell",user)].value != 'Screen name' and sheet[sheet_cell("namecell",user)].value:
        repliesPerParty = allPartiesDict.copy()
        print(str(user+1) + ' - ' + sheet[sheet_cell("namecell",user)].value + ': ' + str(len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['tweet_type'] == 'reply')])))
        if len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['tweet_type'] == 'reply')]) > 0:
            members = {}
            for member in userparties:
                if member.lower() != sheet[sheet_cell("namecell",user)].value.lower():
                    members[member] = len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['in_reply_to_screen_name'] == member) & (data['tweet_type'] == 'reply')])
            for member in members:
                if members[member] > 0:
                    print(member + ' - ' + userparties[member] + ' - ' + str(members[member]))
                    repliesPerParty[userparties[member]] += members[member]
        x = 7
        for i in repliesPerParty:
            x += 1
            partyCell = get_column_letter(x) + str(user + 2)
            replies[partyCell] = repliesPerParty[i]
            print(partyCell + ' - ' + str(repliesPerParty[i]))
            
            
    print('B: ' + str(time.perf_counter() - full_running_time))

finalfile.save(dataSource + 'Seedlist.xlsx')


In [ ]:
### Hashtags - Store hashtags - Hashtags
hashtagstest = []
for i in data[(data['tweet_type'] == 'original')]['hashtags']:
    if isinstance(i,str):
        for z in i.split():
            hashtagstest.append(z)

hashtagtestdict = {}
for i in hashtagstest:
    if i in hashtagtestdict:
        hashtagtestdict[i] += 1
    else:
        hashtagtestdict[i] = 1
        
hashtagsPerUser = {}
for user in range(file_range):
    userHashtags = []
    if sheet[sheet_cell("namecell",user)].value != 'Null' and sheet[sheet_cell("namecell",user)].value != 'Screen name' and sheet[sheet_cell("namecell",user)].value:
        for i in data[(data['tweet_type'] == 'original') & (data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower())]['hashtags']:
            if isinstance(i,str):
                for z in i.split():
                    if z.lower() not in userHashtags:
                        userHashtags.append(z.lower())
    hashtagsPerUser[user] = userHashtags

hashtagsUse = {}
for i in hashtagtestdict:
    x = 0
    hashtagUsers = []
    for z in hashtagsPerUser:
        if i.lower() in hashtagsPerUser[z]:
            x += 1
            hashtagUsers.append(z)
    if len(hashtagUsers) > 19:
        hashtagsUse[i.lower()] = hashtagUsers

x = 8
for i in hashtagsUse:
    hashtags[get_column_letter(x) + str(1)] = i
    for z in hashtagsUse[i]:
        hashtags[get_column_letter(x) + str(z + 2)] = 1
    x += 1

finalfile.save(dataSource + 'Seedlist.xlsx')

In [ ]:
### Collection: No of tweets in weekends - DayOfWeek

# Note: Seems to have some mistake. The data is collected one row above the right one, and so the last user has no data.

for user in range(file_range):
    if sheet[sheet_cell("namecell",user)].value != 'Null' and sheet[sheet_cell("namecell",user)].value != 'Screen name' and sheet[sheet_cell("namecell",user)].value:
        print(str(user) + ' - ' + sheet[sheet_cell("namecell",user)].value)
        for day in range(7):
            dayOfWeek = get_column_letter(day+8) + str(user+2)
            weekDay[dayOfWeek] = len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['DayOfWeek'] == day)])

        
finalfile.save(dataSource + 'Seedlist.xlsx')

In [ ]:
### Collection: No of tweets by time of day - Timing


# Note: Seems to have some mistake. The data is collected one row above the right one, and so the last user has no data.

for user in range(file_range):
    if sheet[sheet_cell("namecell",user)].value != 'Null' and sheet[sheet_cell("namecell",user)].value != 'Screen name' and sheet[sheet_cell("namecell",user)].value:
        print(str(user) + ' - ' + sheet[sheet_cell("namecell",user)].value)
        for hour in range(24):
            hourtohour = get_column_letter(hour+8) + str(user+2)
            timing[hourtohour] = len(data[(data['user_screen_name'] == sheet[sheet_cell("namecell",user)].value.lower()) & (data['tweetHour'] == hour)])

        
finalfile.save(dataSource + 'Seedlist.xlsx')

In [ ]:
### Timer result
print('Full execution time: ' + str(time.perf_counter() - full_running_time))